In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

url_control = '/Users/amirrezakamkar/Desktop/Works_in_progress/Ironhack/5_5th_week/Project/2nd_project/data/clean/combined_data_control.txt'
df_control = pd.read_csv(url_control)
url_test = '/Users/amirrezakamkar/Desktop/Works_in_progress/Ironhack/5_5th_week/Project/2nd_project/data/clean/combined_data_test.txt'
df_test = pd.read_csv(url_test)

In [5]:
# Concatenate the DataFrames
merged_df = pd.concat([df_test, df_control], ignore_index=True)

In [47]:
merged_df.tail()

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation,is_completed
317118,9998346,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:27:40,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,0
317119,9998346,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:26:47,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,0
317120,9998346,292425655_16607136645,189177304_69869411700_783154,step_1,2017-03-29 15:26:02,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,0
317121,9998346,292425655_16607136645,189177304_69869411700_783154,start,2017-03-29 15:25:00,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,0
317122,9998921,960034051_85687824805,378424178_59565976155_8051,start,2017-04-18 11:26:31,15.0,189.0,38.0,F,2.0,30644.10,4.0,7.0,Control,0


In [20]:
merged_df.isnull().sum()
df = merged_df

In [33]:
# test group completion rate (see test group notebook):
# Total number of visitors: 37122
# Number of visitors who reached the 'confirm' step: 21725
# Completion rate: 58.52%

# control group completion rate (see control group notebook): 
# Total number of visitors: 32181
# Number of visitors who reached the 'confirm' step: 16039
# Completion rate: 49.84%

#Null Hypothesis (H0): There is no difference in completion rates between the two groups.
#Alternative Hypothesis (H1): There is a difference in completion rates between the two groups.

# alpha = 0.05

from statsmodels.stats.proportion import proportions_ztest

# n1= test group
# n2= control group

# Define the data
n1 = 37122
n2 = 32181
x1 = 21725
x2 = 16039

# Perform the two-proportion z-test
count = np.array([x1, x2])
nobs = np.array([n1, n2])

z_stat, p_value = proportions_ztest(count, nobs, alternative='two-sided')

# Output the results
print(f"Z-Statistic: {z_stat:.4f}, P-Value: {p_value}")


Z-Statistic: 22.8935, P-Value: 5.392475122751855e-116


In [ ]:
# The P-value is a much smaller than the alpha, so we reject the null hypothesis and conclude that the difference
# in completion rate is statistically significant.

In [57]:
df_test['date_time'] = pd.to_datetime(df_test['date_time'])
df_control['date_time'] = pd.to_datetime(df_control['date_time'])

# Function to calculate time spent between steps for each visitor
def calculate_time_spent(df):
    # Sort by visitor_id and date_time to ensure the time order is correct
    df = df.sort_values(by=['visit_id', 'date_time'])
    
    # Calculate the time spent between consecutive steps for the same visitor
    df['time_spent'] = df.groupby('visit_id')['date_time'].diff().shift(-1)
    
    # Drop the last row for each visitor since they don't have a time_spent value
    df = df.dropna(subset=['time_spent'])
    
    return df

# Apply the function to both dataframes
df_test = calculate_time_spent(df_test)
df_control = calculate_time_spent(df_control)

# Calculate mean time spent per process_step in both test and control groups
mean_time_test = df_test.groupby('process_step')['time_spent'].mean().reset_index()
mean_time_control = df_control.groupby('process_step')['time_spent'].mean().reset_index()

# Now `mean_time_test` and `mean_time_control` contain the mean time spent for each process step
display(mean_time_test)
display(mean_time_control)



,process_step,time_spent
0,confirm,0 days 00:12:42.666666666
1,start,0 days 00:01:13.301240096
2,step_1,0 days 00:01:29.581736909
3,step_2,0 days 00:01:25.708752515
4,step_3,0 days 00:03:38.654109589


,process_step,time_spent
0,confirm,0 days 00:01:40.363636363
1,start,0 days 00:01:24.908042744
2,step_1,0 days 00:01:08.201576576
3,step_2,0 days 00:01:41.449556093
4,step_3,0 days 00:03:15.795238095


In [102]:
#Null hypothesis (H0): The mean time spent by the test group is greater than or equal to the mean time spent by the control group.
#Alternative hypothesis (H1): The mean time spent by the test group is less than the mean time spent by the control group.
# alpha = 0.05

# Step 1: Convert 'date_time' to datetime
df['date_time'] = pd.to_datetime(df['date_time'])

# Step 2: Sort the data for proper time difference calculation
df = df.sort_values(by=['client_id', 'visitor_id', 'visit_id', 'date_time'])

# Step 3: Calculate time spent in seconds for each step
df['time_spent'] = df.groupby(['client_id', 'visitor_id', 'visit_id'])['date_time'].diff().dt.total_seconds()

# Drop NaN values (first step has no previous step for time difference calculation)
df = df.dropna(subset=['time_spent'])

# Step 4: Separate the data for test and control groups
df_test = df[df['Variation'] == 'Test']
df_control = df[df['Variation'] == 'Control']

# Step 5: Initialize a list to store the results of the T-test
t_test_results = []

# Step 6: Perform the T-test for each process_step
for step in df['process_step'].unique():
    # Get the time spent for the test and control group for the current process_step
    test_times = df_test[df_test['process_step'] == step]['time_spent']
    control_times = df_control[df_control['process_step'] == step]['time_spent']
    
    # Perform the Two Sample T-test
    t_stat, p_value = stats.ttest_ind(test_times, control_times, equal_var=False)  # unequal variance assumed

    # For one-sided test, adjust p-value (since we want to test if test group has lower time spent)
    if t_stat < 0:  # If the test statistic is negative, the test group has lower time_spent
        p_value_one_sided = p_value / 2  # Divide the two-tailed p-value by 2 for one-tailed test
    else:
        p_value_one_sided = 1 - (p_value / 2)  # If test_stat is positive, it's not supporting H1

    # Store the result in the list with p_value rounded to 3 decimal places
    t_test_results.append({
        'process_step': step,
        't_statistic': t_stat,
        'p_value': p_value_one_sided,
        'hypothesis': 'Test group has lower time_spent' if p_value_one_sided < 0.05 else 'No significant difference'
    })

# Step 7: Convert the list of results into a DataFrame
t_test_results_df = pd.DataFrame(t_test_results)

# Display the results
t_test_results_df


,process_step,t_statistic,p_value,hypothesis
0,step_2,4.382304,0.999992,No significant difference
1,step_3,0.914169,0.819303,No significant difference
2,confirm,3.864442,0.999934,No significant difference
3,step_1,-1.305479,0.097506,No significant difference
4,start,-1.433428,0.076558,No significant difference


In [96]:
# Convert 'date_time' to datetime
df_test['date_time'] = pd.to_datetime(df['date_time'])

# Sort the data for proper time difference calculation
df_test = df_test.sort_values(by=['client_id', 'visitor_id', 'visit_id', 'date_time'])

# Calculate time spent in seconds for each step
df_test['time_spent'] = df.groupby(['client_id', 'visitor_id', 'visit_id'])['date_time'].diff().dt.total_seconds()

# Drop NaN values (first step has no previous step for time difference calculation)
df_test = df_test.dropna(subset=['time_spent'])
# Group by 'process_step' and calculate the mean time spent
mean_time_per_step = df_test.groupby('process_step')['time_spent'].mean().reset_index()

# Rename columns for clarity
mean_time_per_step.columns = ['process_step', 'mean_time_spent']
mean_time_per_step


/var/folders/kh/bjfqwck131971mz369pxctvm0000gn/T/ipykernel_40153/2412007792.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['date_time'] = pd.to_datetime(df['date_time'])


,process_step,mean_time_spent
0,confirm,189.028455
1,start,146.421687
2,step_1,27.188192
3,step_2,58.720137
4,step_3,81.019305


In [98]:
# Convert 'date_time' to datetime
df_control['date_time'] = pd.to_datetime(df['date_time'])

# Sort the data for proper time difference calculation
df_control = df_control.sort_values(by=['client_id', 'visitor_id', 'visit_id', 'date_time'])

# Calculate time spent in seconds for each step
df_control['time_spent'] = df.groupby(['client_id', 'visitor_id', 'visit_id'])['date_time'].diff().dt.total_seconds()

# Drop NaN values (first step has no previous step for time difference calculation)
df_control = df_control.dropna(subset=['time_spent'])
# Group by 'process_step' and calculate the mean time spent
mean_time_per_step = df_control.groupby('process_step')['time_spent'].mean().reset_index()

# Rename columns for clarity
mean_time_per_step.columns = ['process_step', 'mean_time_spent']
mean_time_per_step


/var/folders/kh/bjfqwck131971mz369pxctvm0000gn/T/ipykernel_40153/270041552.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_control['date_time'] = pd.to_datetime(df['date_time'])


,process_step,mean_time_spent
0,confirm,97.469027
1,start,241.204420
2,step_1,43.468354
3,step_2,15.634409
4,step_3,65.867769
